### Installing PostgreSQL

In [2]:
pip install psycopg2-binary

Note: you may need to restart the kernel to use updated packages.


### Importing PostgreSQL Library

In [5]:
import psycopg2
from sqlalchemy import create_engine
import pandas as pd

### Database Creation

In [8]:
#establishing the connection
conn = psycopg2.connect(
   database="postgres", user='postgres', password='simi280498', host='127.0.0.1', port= '5432'
)
conn.autocommit = True

In [10]:
#Creating a cursor object using the cursor() method
cursor = conn.cursor()

In [12]:
#Droping database if already exists.
cursor.execute("DROP DATABASE IF EXISTS loan_management")

In [14]:
#Preparing query to create a database
sql = '''CREATE database loan_management''';

In [16]:
#Creating a database
cursor.execute(sql)
print("Database created successfully........")

Database created successfully........


In [18]:
#Closing the connection
conn.close()

### Table Creation

In [21]:
#Establishing the connection
conn = psycopg2.connect(
   database="loan_management", user='postgres', password='simi280498', host='127.0.0.1', port= '5432'
)
conn.autocommit = True

In [23]:
#Creating a cursor object using the cursor() method
cursor = conn.cursor()

In [25]:
#Droping BORROWER table if already exists.
cursor.execute("DROP TABLE IF EXISTS borrower")

In [27]:
#Preparaing query to create BORROWER table
sql ='''CREATE TABLE borrower(
   borrower_id CHAR(11) PRIMARY KEY,
   age INT CHECK(age >= 18 AND age <= 70) NOT NULL,
   income INT CHECK(income >= 0) NOT NULL,
   credit_score INT CHECK(credit_score >= 300 AND credit_score <= 850) NOT NULL,
   months_employed INT CHECK(months_employed >= 0) NOT NULL,
   num_credit_lines INT CHECK(num_credit_lines > 0 AND num_credit_lines <= 4) NOT NULL,
   dti_ratio FLOAT CHECK(dti_ratio > 0 AND dti_ratio < 1) NOT NULL,
   education VARCHAR(30) CHECK(education IN ('Bachelor''s', 'Master''s', 'High School', 'PhD')) NOT NULL,
   employment_type VARCHAR(40) CHECK(employment_type IN ('Full-time', 'Unemployed', 'Self-employed', 'Part-time')) NOT NULL,
   marital_status VARCHAR(30) CHECK(marital_status IN ('Divorced', 'Married', 'Single')) NOT NULL,
   income_per_credit_line FLOAT CHECK (income_per_credit_line > 0) NOT NULL,
   has_mortgage INT CHECK(has_mortgage IN (0,1)) NOT NULL,
   has_dependents INT CHECK(has_dependents IN (0,1)) NOT NULL,
   has_cosigner INT CHECK(has_cosigner IN (0,1)) NOT NULL
)'''

In [29]:
#Creating BORROWER table
cursor.execute(sql)
print("Table created successfully........")

Table created successfully........


In [31]:
#Droping LOAN table if already exists.
cursor.execute("DROP TABLE IF EXISTS loan")

In [33]:
#Preparaing query to create LOAN table
sql ='''CREATE TABLE loan(
   loan_id CHAR(10) PRIMARY KEY,
   loan_amount INT CHECK(loan_amount > 0) NOT NULL,
   interest_rate DECIMAL(4,2) CHECK(interest_rate > 0) NOT NULL,
   loan_term INT CHECK(loan_term > 0 AND loan_term % 12 = 0) NOT NULL,
   loan_purpose VARCHAR(40) CHECK(loan_purpose IN ('Other', 'Auto', 'Business', 'Home', 'Education')) NOT NULL,
   defaulter INT CHECK(defaulter IN (0,1)) NOT NULL
)'''

In [35]:
#Creating LOAN table
cursor.execute(sql)
print("Table created successfully........")

Table created successfully........


In [37]:
#Droping LOAN_TO_INCOME table if already exists.
cursor.execute("DROP TABLE IF EXISTS loan_to_income")

In [39]:
#Preparaing query to create LOAN table
sql ='''CREATE TABLE loan_to_income(
   loan_id CHAR(10),
   borrower_id CHAR(11),
   loan_amount_to_income FLOAT CHECK(loan_amount_to_income > 0) NOT NULL,
   PRIMARY KEY (loan_id, borrower_id),
   FOREIGN KEY (borrower_id) REFERENCES borrower(borrower_id),
   FOREIGN KEY (loan_id) REFERENCES loan(loan_id)
)'''

In [41]:
#Creating LOAN table
cursor.execute(sql)
print("Table created successfully........")

Table created successfully........


In [43]:
#Closing the connection
conn.close()

### Data Insertion

In [46]:
dbname = 'loan_management'
user = 'postgres'
password = 'simi280498'
host = '127.0.0.1'
port = '5432'

In [48]:
#Establishing the connection
engine = create_engine(f'postgresql://{user}:{password}@{host}/{dbname}')

#### 1. Inserting data into Borrower Table

In [51]:
df = pd.read_excel('Normalized_dataset.xlsx', sheet_name = 'Borrower Table')

In [53]:
df.head()

,borrower_id,age,income,credit_score,months_employed,num_credit_lines,dti_ratio,education,employment_type,marital_status,income_per_credit_line,has_mortgage,has_dependents,has_cosigner
0,BI38PQUQS96,56,85994,520,80,4,0.44,Bachelor's,Full-time,Divorced,21498.500000,1,1,1
1,BHPSK72WA7R,69,50432,458,15,1,0.68,Master's,Full-time,Married,50432.000000,0,0,1
2,BC1OZ6DPJ8Y,46,84208,451,26,3,0.31,Master's,Unemployed,Divorced,28069.333333,1,1,0
3,BV2KKSFM3UN,32,31713,743,0,3,0.23,High School,Full-time,Married,10571.000000,0,0,0
4,BEY08JDHTZP,60,20437,633,8,4,0.73,Bachelor's,Unemployed,Divorced,5109.250000,0,1,0


In [55]:
df.to_sql('borrower', engine, if_exists='append', index=False)

print("Data inserted successfully!")

Data inserted successfully!


#### 2. Inserting data into Loan Table

In [58]:
df = pd.read_excel('Normalized_dataset.xlsx', sheet_name = 'Loan Table')

In [60]:
df.head()

,loan_id,loan_amount,interest_rate,loan_term,loan_purpose,defaulter
0,I38PQUQS96,50587,15.23,36,Other,0
1,HPSK72WA7R,124440,4.81,60,Other,0
2,C1OZ6DPJ8Y,129188,21.17,24,Auto,1
3,V2KKSFM3UN,44799,7.07,24,Business,0
4,EY08JDHTZP,9139,6.51,48,Auto,0


In [62]:
df.to_sql('loan', engine, if_exists='append', index=False)

print("Data inserted successfully!")

Data inserted successfully!


#### 3. Inserting data into Loan_to_income Table

In [65]:
df = pd.read_excel('Normalized_dataset.xlsx', sheet_name = 'Loan_to_Income Table')

In [67]:
df.head()

,loan_id,borrower_id,loan_amount_to_income
0,I38PQUQS96,BI38PQUQS96,0.588262
1,HPSK72WA7R,BHPSK72WA7R,2.467481
2,C1OZ6DPJ8Y,BC1OZ6DPJ8Y,1.534154
3,V2KKSFM3UN,BV2KKSFM3UN,1.412638
4,EY08JDHTZP,BEY08JDHTZP,0.447179


In [69]:
df.to_sql('loan_to_income', engine, if_exists='append', index=False)

print("Data inserted successfully!")

Data inserted successfully!


In [71]:
engine.dispose()